In [1]:
import os
os.environ["LANGCHAIN_API_KEY"] = "ls__9dad2bff5afc4005bcfddb732f280b72"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "3.conversational_chat_bot"

In [2]:
from langchain.llms import CTransformers

# OpenOrca is a massive library of text from books, articles, and websites.
#Mistral 7B learned from this library to become smarter and more versatile.

model_name = 'TheBloke/Mistral-7B-OpenOrca-GGUF'
model_file = 'mistral-7b-openorca.Q4_K_M.gguf'
model_type = 'mistral'

llm = CTransformers(model="C:/Users/arasu/Workspace/Projects/GenAI/models/Mistral_quantized/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
                    model_type=model_type)

c:\Users\arasu\Workspace\Projects\GenAI\3.conversational_chat_bot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='C:/Users/arasu/Workspace/Projects/GenAI/embeddings/sentence-transformers_all-mpnet-base-v2/')

In [4]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
pdf = PdfReader("employment-agreement2018.pdf")
raw_text = ""
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)        

In [6]:
from langchain.vectorstores import Chroma
db_path = "vector_db"
vector_db = Chroma.from_texts(texts,embeddings,persist_directory=db_path)
vector_db.persist()
vector_db = None

In [9]:
from langchain.vectorstores import Chroma
db_path = "vector_db"
vector_db = Chroma(persist_directory=db_path,embedding_function=embeddings)
retriever = vector_db.as_retriever(search_type="similarity",search_kwargs={"k":2})

In [ ]:
docs = retriever.get_relevant_documents("what is my fixed monthly salary?")
docs

In [11]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
# Instantiate an object of Conversation Buffer Memory
chat_history = ConversationBufferWindowMemory(memory_key="chat_history",
										return_messages=True,k=5

										)
conv_qa = ConversationalRetrievalChain.from_llm(llm=llm,  # USing ChatOpenAI as the LLM
                                                retriever=retriever,  # # set the vectorstore to do similarity search
                                                memory=chat_history,
                                                 # Provide the buffer memory object to pass the conversation history to LLM.
                                               )


In [7]:
conv_qa({"question":"what is my fixed monthly salary?"})

c:\Users\arasu\Workspace\Projects\GenAI\3.conversational_chat_bot\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'what is my fixed monthly salary?',
 'chat_history': [],
 'answer': ' Your fixed monthly salary is INR 3,854,167.'}

In [12]:
conv_qa({"question":"am i eligible for a Superannuation?"})

{'question': 'am i eligible for a Superannuation?',
 'chat_history': [],
 'answer': ' Yes, based on the context provided, you are eligible for a Superannuation with a contribution of up to INR 8,333 per month.'}

In [13]:
conv_qa({"question":"what about the balance?"})

{'question': 'what about the balance?',
 'chat_history': [HumanMessage(content='what is my fixed monthly salary?'),
  AIMessage(content=' Your fixed monthly salary is INR 3,854,167.'),
  HumanMessage(content='am i eligible for a Superannuation?'),
  AIMessage(content=" Yes, based on the information provided in the text, an employee with a fixed monthly salary of INR 3,854,167 is eligible for a Superannuation contribution as per the company's policy, which is calculated at 15% of Basic Salary up to a maximum contribution of INR 8,333 per month.")],
 'answer': ' The remaining portion of your salary above INR 8,333 will be paid to you as part of your monthly salary.'}

In [14]:
conv_qa({"question":"am i eligible for a group health insurance?"})

{'question': 'am i eligible for a group health insurance?',
 'chat_history': [HumanMessage(content='what is my fixed monthly salary?'),
  AIMessage(content=' Your fixed monthly salary is INR 3,854,167.'),
  HumanMessage(content='am i eligible for a Superannuation?'),
  AIMessage(content=" Yes, based on the information provided in the text, an employee with a fixed monthly salary of INR 3,854,167 is eligible for a Superannuation contribution as per the company's policy, which is calculated at 15% of Basic Salary up to a maximum contribution of INR 8,333 per month."),
  HumanMessage(content='what about the balance?'),
  AIMessage(content=' The remaining portion of your salary above INR 8,333 will be paid to you as part of your monthly salary.')],
 'answer': ' You are eligible to participate in the Group Health Insurance Scheme provided by Infosys. The default coverage includes you and your family (spouse and two children up to the age of 22 years) with a cover of INR 400,000 per annum. Y

In [15]:
conv_qa({"question":"can you give more details about it?"})

{'question': 'can you give more details about it?',
 'chat_history': [HumanMessage(content='what is my fixed monthly salary?'),
  AIMessage(content=' Your fixed monthly salary is INR 3,854,167.'),
  HumanMessage(content='am i eligible for a Superannuation?'),
  AIMessage(content=" Yes, based on the information provided in the text, an employee with a fixed monthly salary of INR 3,854,167 is eligible for a Superannuation contribution as per the company's policy, which is calculated at 15% of Basic Salary up to a maximum contribution of INR 8,333 per month."),
  HumanMessage(content='what about the balance?'),
  AIMessage(content=' The remaining portion of your salary above INR 8,333 will be paid to you as part of your monthly salary.'),
  HumanMessage(content='am i eligible for a group health insurance?'),
  AIMessage(content=' You are eligible to participate in the Group Health Insurance Scheme provided by Infosys. The default coverage includes you and your family (spouse and two child